# Base loads

In [1]:
!pip install pyro-ppl==1.3.0

     |████████████████████████████████| 501kB 2.7MB/s 


In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
# "Standard" imports
import numpy as np
from time import time
import itertools
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import folium
import datetime

# Pytorch imports
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

# Pyro imports
import pyro
from pyro.distributions import ConditionalTransformedDistribution, ConditionalTransformModule, TransformModule
import pyro.distributions as dist
from pyro.distributions.transforms import affine_coupling, affine_autoregressive, permute

# Sklearn imports
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# Notebooks imports
from IPython.display import Image, display, clear_output
from tqdm import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual


import numpy as np
import folium
from folium import plugins
import matplotlib
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import drive
import sys
import os
drive.mount('/content/drive/')
root_path = 'drive/My Drive/Colab_Notebooks/normalizingflows'
trained_flows_folder = 'drive/My Drive/Colab_Notebooks/normalizingflows/trained_flows'
dataset_folder = 'drive/My Drive/Colab_Notebooks/normalizingflows/datasets'
sys.path.append(root_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Load Git code

In [5]:
%cd drive/'My Drive'/Thesis/code/DEwNF/
!git pull
%cd /content/

/content/drive/My Drive/Thesis/code/DEwNF
Already up to date.
/content


In [0]:
git_folder_path = 'drive/My Drive/Thesis/code/DEwNF/'
sys.path.append(git_folder_path)

In [0]:
from DEwNF.flows import ConditionalAffineCoupling, ConditionedAffineCoupling, ConditionalNormalizingFlowWrapper, conditional_affine_coupling, normalizing_flow_factory, conditional_normalizing_flow_factory
from DEwNF.utils import plot_4_contexts_cond_flow, plot_loss, sliding_plot_loss, plot_samples
from DEwNF.samplers import RotatingTwoMoonsConditionalSampler
from DEwNF.regularizers import NoiseRegularizer, rule_of_thumb_noise_schedule, approx_rule_of_thumb_noise_schedule, square_root_noise_schedule, constant_regularization_schedule


## Load datasets

In [0]:
donkey_folder = os.path.join(dataset_folder, 'Donkey_republic')
csv_path = os.path.join(donkey_folder, 'CoordinateSearchlog2.csv')
donkey_df = pd.read_csv(csv_path, parse_dates=[4, 11])


In [0]:
hubs_path = os.path.join(donkey_folder, "Hubs.csv")
hubs_df = pd.read_csv(hubs_path, parse_dates=[0])
hubs_df['deleted'] = ~hubs_df.deleted_at.isnull()


In [0]:
hubs_ranking_path = os.path.join(donkey_folder, 'HubsRankedByRentals.csv')
ranked_hubs_df = pd.read_csv(hubs_ranking_path, index_col=0)


In [0]:
coordinate_hubs_path = os.path.join(donkey_folder, 'CoordinateHubs.csv')
hub_rentals_df = pd.read_csv(coordinate_hubs_path)

In [0]:
# Define scalers
obs_cols = ['user_location_latitude', 'user_location_longitude']
obs_scaler = StandardScaler().fit(donkey_df.loc[:, obs_cols])

In [15]:
hubs_df.dropna()

,created_at,latitude,longitude,id,name,deleted_at,deleted
0,2018-05-21 09:35:00.697175,48.864936,2.310624,3268,Cours la Reine,2018-05-21 16:17:24.722283,True
2,2018-06-05 12:12:28.639837,55.676916,12.564896,3642,Concert Hall Pumpehuset,2018-11-15 10:45:34.468962,True
16,2018-01-22 15:47:39.318344,55.690060,12.554953,2378,Solitudvej,2018-12-03 15:52:51.1378,True
19,2018-04-15 09:19:30.266989,55.683848,12.562526,2850,Finsensvej,2018-11-07 14:29:06.066021,True
21,2017-07-05 08:47:19.611345,55.686668,12.585904,1521,Adelgade,2018-12-12 18:28:38.467796,True
...,...,...,...,...,...,...,...
745,2018-10-31 10:17:31.024599,55.679149,12.585979,6166,Den Kongelige Teater,2018-11-15 10:48:58.259482,True
912,2018-10-30 09:40:07.236571,55.673243,12.571512,6162,Tivoli Corner,2018-11-15 11:47:56.897256,True
922,2018-07-11 10:43:36.716567,55.649542,12.555709,4771,Teglholmens Ã˜stkaj,2018-12-19 18:21:08.468732,True
931,2018-04-15 07:43:13.731163,55.658776,12.601904,2833,Sundholmsvej 79,2018-09-25 12:39:15.800906,True


In [16]:
hubs_df

,created_at,latitude,longitude,id,name,deleted_at,deleted
0,2018-05-21 09:35:00.697175,48.864936,2.310624,3268,Cours la Reine,2018-05-21 16:17:24.722283,True
1,2018-11-26 09:06:05.590590,55.695252,12.547185,6367,Heinesgade,NaN,False
2,2018-06-05 12:12:28.639837,55.676916,12.564896,3642,Concert Hall Pumpehuset,2018-11-15 10:45:34.468962,True
3,2018-05-28 20:07:34.173984,55.668700,12.551114,3526,Saxogade,NaN,False
4,2018-11-08 09:36:50.440822,55.699557,12.515065,6233,GrÃ¸ndal Multicenter,NaN,False
...,...,...,...,...,...,...,...
1261,2018-11-15 12:56:25.471178,55.694262,12.570220,6297,Trepkasgade,NaN,False
1262,2016-02-16 11:55:37.506379,55.682597,12.571227,60,NÃ¸rreport St. (Depot),NaN,False
1263,2018-06-23 09:33:34.977976,55.631240,12.580264,4149,Ã˜restad Bibliotek,NaN,False
1264,2018-06-14 09:17:33.418759,55.644050,12.545233,3757,Sluseholmen,NaN,False


In [17]:
1266 - 119

1147

## Plotting setup


In [0]:
plot_folder = f'{root_path}/plot_notebooks/generated_plots'

## Closeup heatmaps

In [0]:

color_dict = {True: '#3186cc', False:'crimson'}


In [0]:
def add_hub_markers(df, columns, folium_plot, opacity=0.2):
    for point in range(0, len(df)):
        row = df.loc[point]
        folium.CircleMarker(
        location=row[columns].values,
        radius=0.5,
        popup=f"Hub_{row['id']}",
        color=color_dict[row['deleted']],
        fill=True,
        fill_color=color_dict[row['deleted']], opacity=opacity).add_to(folium_plot)

### Searchlogs

In [0]:
obs_cols = ['user_location_latitude', 'user_location_longitude']
xy_data = donkey_df.loc[:,obs_cols].values[100000:]
#initial_location = [55.6775757, 12.579571639999999]
initial_location = [55.683167, 12.571389]
hub_map = folium.Map(width=500,height=500,location=initial_location, zoom_start=16, zoom_control=False)
#plugins.HeatMap(xy_data, radius=10, blur=25).add_to(hub_map)
plugins.HeatMap(xy_data, radius=5, blur=10).add_to(hub_map)
add_hub_markers(hubs_df, ['latitude', 'longitude'], hub_map)
display(hub_map)
hub_map.save(f'{plot_folder}/searchlog_closeup_heatmap.html')

### hub rentals

In [0]:
obs_cols = ['pickup_latitude', 'pickup_longitude']
xy_data =  hub_rentals_df.loc[:,obs_cols].values[:100000]
small_noise = np.random.normal(np.zeros(xy_data.shape), 1e-5*np.ones(xy_data.shape))
xy_data = xy_data + small_noise
#initial_location = [55.6775757, 12.579571639999999]
initial_location = [55.683167, 12.571389]
hub_map = folium.Map(width=500,height=500,location=initial_location, zoom_start=16, zoom_control=False)
plugins.HeatMap(xy_data, radius=5, blur=10).add_to(hub_map)
#add_hub_markers(hubs_df, ['latitude', 'longitude'], hub_map)
display(hub_map)
hub_map.save(f'{plot_folder}/rentals_closeup_heatmap.html')

## Overview heatmap of searchlogs

In [0]:
obs_cols = ['user_location_latitude', 'user_location_longitude']
xy_data = donkey_df.loc[:,obs_cols].values[100000:]
initial_location = [55.6775757, 12.579571639999999]
hub_map = folium.Map(width=500,height=500,location=initial_location, zoom_start=11, zoom_control=False)
plugins.HeatMap(xy_data, radius=10, blur=25).add_to(hub_map)
#add_hub_markers(hubs_df, ['latitude', 'longitude'], hub_map)
display(hub_map)
hub_map.save(f'{plot_folder}/searchlog_overview_heatmap.html')

## Over view of hubs in Copenhagen

In [0]:
obs_cols = ['user_location_latitude', 'user_location_longitude']
xy_data = donkey_df.loc[:,obs_cols].values[100000:]
initial_location = [55.6775757, 12.579571639999999]
hub_map = folium.Map(width=500,height=500,location=initial_location, zoom_start=11, zoom_control=False)
#plugins.HeatMap(xy_data, radius=10, blur=25).add_to(hub_map)
add_hub_markers(hubs_df, ['latitude', 'longitude'], hub_map, opacity=1)
display(hub_map)
hub_map.save(f'{plot_folder}/hubs_overview.html')